In [1]:
%load_ext jupyter_black

In [2]:
import os
from shapely.geometry import Polygon

os.environ["USE_PYGEOS"] = "0"
import geopandas
import shutil
import numpy as np
import napari
import os
import spatialdata as sd
from spatialdata.transformations import (
    Affine,
    Sequence,
    Identity,
    align_elements_using_landmarks,
    get_transformation,
    set_transformation,
)
from napari_spatialdata import Interactive

In [3]:
print("current working directory:", os.getcwd())
SPATIALDATA_SANDBOX_PATH = "spatialdata-sandbox"
assert os.path.isdir(
    SPATIALDATA_SANDBOX_PATH
), f"{SPATIALDATA_SANDBOX_PATH} not found, please use symlinks to make it available"
GENERATED_DATA_PATH = os.path.join(
    SPATIALDATA_SANDBOX_PATH, "generated_data/xenium_visium_integration"
)
assert os.path.isdir(
    GENERATED_DATA_PATH
), f"{GENERATED_DATA_PATH} not found, please use symlinks to make it available"


XE_REP1_PATH = os.path.join(SPATIALDATA_SANDBOX_PATH, "xenium_rep1_io/data.zarr")
XE_REP2_PATH = os.path.join(SPATIALDATA_SANDBOX_PATH, "xenium_rep2_io/data.zarr")
VISIUM_PATH = os.path.join(
    SPATIALDATA_SANDBOX_PATH, "visium_associated_xenium_io/data.zarr"
)

XE_REP1_ROI_PATH = os.path.join(GENERATED_DATA_PATH, "xe_rep1_roi.zarr")
XE_REP2_ROI_PATH = os.path.join(GENERATED_DATA_PATH, "xe_rep2_roi.zarr")
VISIUM_ROI_PATH = os.path.join(GENERATED_DATA_PATH, "visium_roi.zarr")

LANDMARKS_SDATA_PATH = os.path.join(GENERATED_DATA_PATH, "sandbox.zarr")

current working directory: /Users/macbook/embl/projects/basel/spatialdata-sandbox/notebooks


In [4]:
# we delete some elements (in-memory only) because we will not need them in napari
xe_rep1_sdata = sd.read_zarr(XE_REP1_PATH)
del xe_rep1_sdata.images["morphology_focus"]
del xe_rep1_sdata.shapes["cell_boundaries"]
del xe_rep1_sdata.shapes["nucleus_boundaries"]
xe_rep2_sdata = sd.read_zarr(XE_REP2_PATH)
del xe_rep2_sdata.images["morphology_focus"]
del xe_rep2_sdata.shapes["cell_boundaries"]
del xe_rep2_sdata.shapes["nucleus_boundaries"]
visium_sdata = sd.read_zarr(VISIUM_PATH)

xe_rep1_roi_sdata = sd.read_zarr(XE_REP1_ROI_PATH)
del xe_rep1_roi_sdata.shapes["cell_boundaries"]
del xe_rep1_roi_sdata.shapes["nucleus_boundaries"]
xe_rep2_roi_sdata = sd.read_zarr(XE_REP2_ROI_PATH)
del xe_rep2_roi_sdata.shapes["cell_boundaries"]
del xe_rep2_roi_sdata.shapes["nucleus_boundaries"]
visium_roi_sdata = sd.read_zarr(VISIUM_ROI_PATH)

landmarks_sdata = sd.read_zarr(LANDMARKS_SDATA_PATH)

INFO     Instance key `cell_id` could be of type `pd.Categorical`. Consider casting it.                            
INFO     Instance key `cell_id` could be of type `pd.Categorical`. Consider casting it.                            


In [5]:
landmarks_sdata["rois"]["name"] = [
    "Invasive",
    "Immune infiltration",
    "DCIS",
    "Necrotic core",
]
# the new extra column gets not saved to Zarr, we need to discuss this behavior (and the relationships with multiple tables) in the next hackathon
landmarks_sdata["rois"]

,geometry,name
0,"POLYGON ((1928.166 9124.230, 1928.166 9151.215...",Invasive
1,"POLYGON ((8109.926 14163.968, 8109.926 14173.7...",Immune infiltration
2,"POLYGON ((32042.805 22387.401, 32026.302 22436...",DCIS
3,"POLYGON ((27504.676 20232.314, 27514.049 20237...",Necrotic core


In [6]:
# remove the cells, for xenium rep 2, inside the box (so already present in xenium rep 1)
xe_rep2_sdata.shapes["cell_circles"] = xe_rep2_sdata["cell_circles"][
    ~xe_rep2_sdata["cell_circles"].index.isin(xe_rep2_roi_sdata["cell_circles"].index)
]

In [9]:
from anndata import AnnData


def set_colors_for_categorical_obs(
    adata: AnnData, column: str, colors_dict: dict[str, str]
):
    colors = []
    for cat in adata.obs[column].cat.categories.tolist():
        color = colors_dict[cat]
        colors.append(color)
    adata.uns[f"{column}_colors"] = colors


celltype_major_colors = {
    "B-cells": "#d8f55e",
    "CAFs": "#532C8A",
    "Cancer Epithelial": "#C72228",
    "Endothelial": "#9e6762",
    "Myeloid": "#ffe012",
    "T-cells": "#3cb44b",
    "Normal Epithelial": "#0F4A9C",
    "PVL": "#c09d9a",
    "Plasmablasts": "#000075",
}
set_colors_for_categorical_obs(
    adata=xe_rep1_sdata.table,
    column="celltype_major",
    colors_dict=celltype_major_colors,
)
set_colors_for_categorical_obs(
    adata=xe_rep2_sdata.table,
    column="celltype_major",
    colors_dict=celltype_major_colors,
)

In [105]:
xe_rep1_roi_sdata.table.obs["celltype_major"] = xe_rep1_roi_sdata.table.obs[
    "celltype_major"
].astype("category")
xe_rep1_roi_sdata.table.obs["celltype_minor"] = xe_rep1_roi_sdata.table.obs[
    "celltype_minor"
].astype("category")
xe_rep2_roi_sdata.table.obs["celltype_major"] = xe_rep2_roi_sdata.table.obs[
    "celltype_major"
].astype("category")
xe_rep2_roi_sdata.table.obs["celltype_minor"] = xe_rep2_roi_sdata.table.obs[
    "celltype_minor"
].astype("category")

set_colors_for_categorical_obs(
    adata=xe_rep1_roi_sdata.table,
    column="celltype_major",
    colors_dict=celltype_major_colors,
)
set_colors_for_categorical_obs(
    adata=xe_rep2_roi_sdata.table,
    column="celltype_major",
    colors_dict=celltype_major_colors,
)

In [ ]:
Interactive([xe_rep1_sdata, xe_rep2_sdata, visium_sdata, landmarks_sdata], points=False)

In [46]:
from spatialdata import SpatialData
from shapely import Polygon
from spatialdata.models import ShapesModel


# same function of notebooks 00, move to a .py file, later on include a reworked version in spatialdata
def spatial_query_shapes_and_points_by_polygon(
    sdata: SpatialData, polygon: Polygon, target_coordinate_system: str
):
    new_shapes = {}
    for shapes_name, shapes in sdata.shapes.items():
        if "__old_index" in shapes.columns:
            assert np.all(shapes["__old_index"] == shapes.index)
        else:
            shapes["__old_index"] = shapes.index
        # indices = shapes.intersects(polygon)
        # indices = shapes.geometry.apply(lambda x: x.intersects(polygon).values.tolist()[0][0])
        indices = shapes.geometry.apply(lambda x: x.intersects(polygon))
        if np.sum(indices) == 0:
            raise ValueError("we expect at least one shape")
        queried_shapes = shapes[indices]
        queried_shapes.index = queried_shapes["__old_index"]
        del shapes["__old_index"]
        del queried_shapes["__old_index"]
        transformation = get_transformation(shapes, target_coordinate_system)
        queried_shapes = ShapesModel.parse(queried_shapes)
        set_transformation(queried_shapes, transformation, target_coordinate_system)
        new_shapes[shapes_name] = queried_shapes

    new_points = {}
    for points_name, points in sdata.points.items():
        # let's ignore the z component here
        points_gdf = points_dask_dataframe_to_geopandas(points)
        # queried_points = points_gdf.intersects(pp)
        indices = points_gdf.geometry.intersects(polygon)
        if np.sum(indices) == 0:
            raise ValueError("we expect at least one point")
        queried_points = points_gdf[indices]
        ddf = points_geopandas_to_dask_dataframe(queried_points)
        new_points[points_name] = ddf

    return SpatialData(shapes=new_shapes, points=new_points, table=sdata.table)

In [109]:
import pandas as pd
from spatialdata.models import TableModel

rois = landmarks_sdata["rois"]
sdatas = []

for roi in rois.iterrows():
    geometry, name = roi[1]
    sdata = spatial_query_shapes_and_points_by_polygon(
        SpatialData(shapes={"cell_circles": xe_rep1_roi_sdata["cell_circles"]}),
        geometry,
        "aligned",
    )
    region_name = f"Cells in ROI {name!r}"
    merged = pd.merge(
        xe_rep1_roi_sdata.table.obs[["cell_id"]],
        sdata.shapes["cell_circles"],
        how="right",
        left_on="cell_id",
        right_index=True,
    )
    filtered_table = xe_rep1_roi_sdata.table[merged.index].copy()
    filtered_table.obs["region"] = region_name
    del filtered_table.uns["spatialdata_attrs"]
    TableModel.parse(
        filtered_table, region=region_name, region_key="region", instance_key="cell_id"
    )

    sdata = SpatialData(
        shapes={region_name: sdata["cell_circles"]}, table=filtered_table
    )
    sdatas.append(sdata)

cells_in_rois_sdata = sd.concatenate(sdatas)

cells_in_rois_sdata.table.obs["celltype_major"] = cells_in_rois_sdata.table.obs[
    "celltype_major"
].astype("category")
set_colors_for_categorical_obs(
    adata=cells_in_rois_sdata.table,
    column="celltype_major",
    colors_dict=celltype_major_colors,
)

/tmp/ipykernel_71191/1975229073.py:25: UserWarning: Converting `region_key: region` to categorical dtype.
  TableModel.parse(
/tmp/ipykernel_71191/1975229073.py:25: UserWarning: Converting `region_key: region` to categorical dtype.
  TableModel.parse(
/tmp/ipykernel_71191/1975229073.py:25: UserWarning: Converting `region_key: region` to categorical dtype.
  TableModel.parse(
/tmp/ipykernel_71191/1975229073.py:25: UserWarning: Converting `region_key: region` to categorical dtype.
  TableModel.parse(


In [115]:
visium_sdata

SpatialData object with:
├── Images
│     ├── 'CytAssist_FFPE_Human_Breast_Cancer_full_image': MultiscaleSpatialImage[cyx] (3, 21571, 19505), (3, 10785, 9752), (3, 5392, 4876), (3, 2696, 2438), (3, 1348, 1219)
│     ├── 'CytAssist_FFPE_Human_Breast_Cancer_hires_image': SpatialImage[cyx] (3, 2000, 1809)
│     └── 'CytAssist_FFPE_Human_Breast_Cancer_lowres_image': SpatialImage[cyx] (3, 600, 543)
├── Shapes
│     └── 'CytAssist_FFPE_Human_Breast_Cancer': GeoDataFrame shape: (4992, 2) (2D shapes)
└── Table
      └── AnnData object with n_obs × n_vars = 4992 × 18085
    obs: 'in_tissue', 'array_row', 'array_col', 'spot_id', 'region', 'dataset'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial', 'spatialdata_attrs', 'region_colors'
    obsm: 'spatial': AnnData (4992, 18085)
with coordinate systems:
▸ 'aligned', with elements:
        CytAssist_FFPE_Human_Breast_Cancer_full_image (Images), CytAssist_FFPE_Human_Breast_Cancer (Shapes)
▸ 'downscaled_hires', with elements:
        

In [ ]:
Interactive(
    [
        SpatialData(images={"morphology_mip": xe_rep1_sdata["morphology_mip"]}),
        SpatialData(images={"morphology_mip": xe_rep2_sdata["morphology_mip"]}),
        SpatialData(
            images={
                "CytAssist_FFPE_Human_Breast_Cancer_full_image": visium_sdata[
                    "CytAssist_FFPE_Human_Breast_Cancer_full_image"
                ]
            }
        ),
        landmarks_sdata,
        cells_in_rois_sdata,
    ],
    points=False,
)

In [119]:
# rois colors (this is the order how the ROIs are saved)
rois_colors = {
    "Invasive": "#4cb1ff",
    "Immune infiltration": "#7dc573",
    "Necrotic core": "#ffd700",
    "DCIS": "#ff5500",
}